In [1]:
# https://huggingface.co/docs/transformers/model_doc/gptj

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from time import time

DEVICE = "cuda"

# tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B")
# model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B",
#                                             pad_token_id=tokenizer.pad_token_id,
#                                             eos_token_id=tokenizer.eos_token_id,
#                                             low_cpu_mem_usage=True,
# #                                             torch_dtype='auto'
#                                             torch_dtype=torch.float16
#                                             ).to(DEVICE)

tokenizer = AutoTokenizer.from_pretrained(
    "kakaobrain/kogpt",
    revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
    bos_token="[BOS]",
    eos_token="[EOS]",
    unk_token="[UNK]",
    pad_token="[PAD]",
    mask_token="[MASK]",
)
model = AutoModelForCausalLM.from_pretrained(
    './kogpt-ft-3',
#     "kakaobrain/kogpt",
    revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
    pad_token_id=tokenizer.pad_token_id,
    torch_dtype=torch.float16,
    # torch_dtype='auto',
    low_cpu_mem_usage=True,
).to(DEVICE)
_ = model.eval()

# prompt = "원숭이 엉덩이는 빨개. 빨가면 사과."


# st = time()
# with torch.no_grad():
#     tokens = tokenizer.encode(prompt, return_tensors="pt").to(
#         device=DEVICE, non_blocking=True
#     )
#     gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
#     generated = tokenizer.batch_decode(gen_tokens)[0]

# end = time()
# print(f"[Elpsed]: {end-st} sec")

# print(generated)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# PROMPT_DICT = {
#     "prompt_input": (
#         "Below is an instruction that describes a task, paired with an input that provides further context.\n"
#         "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
#         "Write a detailed response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 자세하게 작성하세요.\n\n"
#         "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{user_input}\n\n### Response(응답):"
#     ),
#     "prompt_no_input": (
#         "Below is an instruction that describes a task.\n"
#         "아래는 작업을 설명하는 명령어입니다.\n\n"
#         "Write a detailed response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 자세하게 작성하세요.\n\n"
#         "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
#     ),
# }


PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n\n"
        "Write a detailed response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{user_input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n\n"
        "Write a detailed response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

def gen(prompt, user_input=None, min_new_tokens=10, max_new_tokens=1024, temperature=0.5):
    st = time()
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, user_input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to(DEVICE)
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens,
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
        
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    end = time()
#     print(f"[Elpsed]: {end-st} sec")
    
    return x, gen_text.replace(x, '')

In [ ]:
for i in range(5):
    prompt, generated_ouput = gen('직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 작성해줘.', max_new_tokens=1024, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

In [ ]:
for i in range(5):
    prompt, generated_ouput = gen('한국에서 제일 유명한 가수와 그 대표곡을 알려줘.', max_new_tokens=1024, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

In [3]:
for i in range(5):
    prompt, generated_ouput = gen('농심에서 만든 라면 5가지와 그 특성을 알려줘.', max_new_tokens=1024, temperature=1.5)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

Below is an instruction that describes a task.

Write a detailed response that appropriately completes the request.

### Instruction:
농심에서 만든 라면 5가지와 그 특성을 알려줘.

### Response: 
 국내 인기 농심 라면 5종은 너구리, 오징어김치, 차돌과 너구리, 쌀국수볶음너구리, 사천짜파게티입니다. 예로부터 맛있기로 유명한 너구리와 볶음너구리, 짜파게티를 비롯해 다양한 종류로 만들어져 선택의 폭이 넓습니다. 건강을 중요시하는 요즘 트렌드에 맞춘 차돌과 너구리 또한 판매량이 높은 제품입니다.

농심에서 만든 라면은 너구리, 스낵면, 매운 너구리, 김치면, 육개탕 등입니다. 각 제품은 고유한 맛을 가지고 있고, 전체적으로 약간의 특별한 향이 있습니다. 또한, 전체적으로 더 건강한 식사를 위해 대체 육류나 채소를 첨부하기도 합니다.

아래는 참고하시라고 5가지 라면 특성을 정리한 내용입니다.

- 가장 칼로리가 낮음(가장 청양한 라면으로 고소한 맛을 냄)
- 국물의 염도가 낮음
- 면과 수프의 텐션이 높음
- 조리 시 계란을 하나 더 추가해도 충분한 단백질 함량 보강 가능
- 국물이 쉽게 베어 짠맛을 줄일 수 있는 효과적인 염도 적용

농심에서 만든 대표적인 라면은 매운 너구리, 스낵면, 열무 비빔면, 까르보나라 불닭볶음면, 후루코 등입니다. 각각 다른 맛과 특성을 지니고 있습니다.

농심에서 만드는 라면은 열무김치면, 채식라면, 차돌박이라면, 감자면, 오동통한 너구리 곰탕라면입니다. 이중 열무김치면은 여름철에 알맞은 비빔면으로 만들어 먹으면 정말 맛있습니다. 또한, 차돌박이 칼국수, 감자 라면, 너구리 곰탕 라면을 만들 수 있습니다.



---
## MAKE DATA

In [4]:
import json
from tqdm.auto import tqdm

with open('./ko_alpaca_data.json', 'r') as f:
    data = json.load(f)

In [5]:
len(data)

49620

In [12]:
# make exist file as backup file
dfile_name = "rlhf-data.jsonl"
backup_name = "rlhf-data.old.jsonl"
if os.path.exists(dfile_name):
    if os.path.exists(backup_name):
        raise Exception(f'백업파일({backup_name}) 이 이미 존재합니다. 기존 백업 파일명을 변경해주세요')
    os.rename(dfile_name, backup_name)

In [20]:
new_data = []
for d in tqdm(data):
    res = {}
    prompt, user_input, chatgpt_outpout = d['instruction'], d['input'], d['output']    
    res.update({'instruction': prompt, 'input': user_input, 'output': [chatgpt_outpout]})
    if user_input == '':
        user_input = None
    for i in range(2):
        final_prompt, generated_ouput = gen(prompt, user_input=user_input, max_new_tokens=1024, temperature=1.5)
        res['output'].append(generated_ouput.strip())
    new_data.append(res)
#     sleep(0.5)

    if len(new_data) == 50:
        with open(dfile_name, 'a') as f:
            print('write data:', len(new_data))
            dl = [json.dumps(x, ensure_ascii=False) for x in new_data]
            f.write('\n'.join(dl) + '\n')
            new_data = []
            

  0%|          | 0/49620 [00:00<?, ?it/s]

write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2
write data: 2


KeyboardInterrupt: 